### Irrigation model input file prep

This code prepares the final input file to the irrigation (agrodem) model. It extracts all necessary attributes to crop locations. It also applies some name fixes as needed for the model to run smoothly.The output dataframe is exported as csv and ready to be used in the irrigation model.

**Original code:** [Alexandros Korkovelos](https://github.com/akorkovelos) & [Konstantinos Pegios](https://github.com/kopegios)<br />
**Conceptualization & Methodological review :** [Alexandros Korkovelos](https://github.com/akorkovelos)<br />
**Updates, Modifications:** [Alexandros Korkovelos](https://github.com/akorkovelos)<br />
**Funding:** The World Bank (contract number: 7190531), [KTH](https://www.kth.se/en/itm/inst/energiteknik/forskning/desa/welcome-to-the-unit-of-energy-systems-analysis-kth-desa-1.197296)

In [7]:
#Import modules and libraries
import os
import geopandas as gpd
from rasterstats import point_query
import logging
import pandas as pd
from shapely.geometry import Point, Polygon
import gdal
import rasterio as rio
import fiona
import gdal
import osr
import ogr
import rasterio.mask
import time
import numpy as np
import itertools
import re
from osgeo import gdal,ogr
import struct
import csv
import tkinter as tk
from tkinter import filedialog, messagebox
from pandas import DataFrame as df
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterstats import point_query
from pyproj import Proj
from shapely.geometry import Point, Polygon

In [12]:
# Import data 

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)
messagebox.showinfo('Agrodem Prepping', 'Open the extracted csv file obtained after running the QGIS plugin - surface water extractor')
input_file = filedialog.askopenfilename()

# Import csv as pandas dataframe
crop_df = pd.read_csv(input_file)

In [13]:
# Fill in Nan values with 0
crop_df.fillna(99999,inplace=True)
crop_df.head(2)

,country_co,country,admin2,_mean,alloc_key,cell5m,x,y,harea_spam,name_cntr,shareofsum,harv_area_,district_a,shareofdis,_mean_1
0,SOM,Somalia,Baki,256.933333,9602681,4145560,43.375000,10.041667,1481.0,Somalia,1.476108e-02,1371.599719,369427.253525,3.712774e-03,565.677688
1,SOM,Somalia,Baki,256.933333,9602682,4145561,43.458333,10.041667,0.1,Somalia,9.966969e-07,0.092613,369427.253525,2.506937e-07,565.677688


In [14]:
crop_df.columns

Index(['country_co', 'country', 'admin2', '_mean', 'alloc_key', 'cell5m', 'x',
       'y', 'harea_spam', 'name_cntr', 'shareofsum', 'harv_area_',
       'district_a', 'shareofdis', '_mean_1'],
      dtype='object')

In [18]:
#somalia only
crop_df=crop_df.drop(['district_a', 'shareofdis','_mean', 'alloc_key', 'cell5m','country_co', 'country','name_cntr', 'shareofsum', 'harea_spam'],axis = 1)

In [19]:
#somalia only
crop_df.rename(columns={'_mean_1': 'sw_depth',
                         'harv_area_': 'harv_area',
                       'admin2':'state',
                       'x':'lon',
                       'y':'lat'}, inplace=True)

In [20]:
crop_df.columns

Index(['state', 'lon', 'lat', 'harv_area', 'sw_depth'], dtype='object')

In [6]:
##Dropping unecessary columns

droping_cols = ['Pixel']
crop_df.drop(droping_cols, axis=1)

# New for whole Moz
crop_df.rename(columns={'elevation': 'sw_depth',
                         'CropArea': 'harv_area'}, inplace=True)

# Adding columns missing
crop_df["country"] = "Somalia"
crop_df["curr_yield"] = "580"
crop_df["max_yield"] = "1493"
crop_df['field_1'] = range(0, 0+len(crop_df))

KeyError: "['Pixel'] not found in axis"

In [21]:
#Somalia
#Adding columns missing
crop_df["country"] = "Somalia"
crop_df["curr_yield"] = "580"
crop_df["max_yield"] = "1493"
crop_df['field_1'] = range(0, 0+len(crop_df))

#### Converting dataframe to geo-dataframe

In [22]:
# Add geometry and convert to spatial dataframe in source CRS
crop_df['geometry'] = list(zip(crop_df['lon'], crop_df['lat']))
crop_df['geometry'] = crop_df['geometry'].apply(Point)
crop_df = gpd.GeoDataFrame(crop_df, geometry='geometry', crs={'init': 'epsg:4326'})

#Reproject data in to Ordnance Survey GB coordinates
crop_df = crop_df.to_crs({'init': 'epsg:3395'})

C:\Users\oluchi\.conda\envs\agrodem_run\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [23]:
# convert to shapefile
#write the name you would like to have in the string "test_final5, you can keep this also as the default name"
crop_df.to_file('Maize_Somalia_10km.shp',driver = 'ESRI Shapefile')

In [ ]:
#export to csv
messagebox.showinfo('Agrodem Prepping','Browse to the folder where you want to save geodataframe as a csv file')
path = filedialog.askdirectory()
shpname = 'Maize_Somalia_10km'
crop_df.to_csv(os.path.join(path,"{}.csv".format(shpname)))

In [9]:
messagebox.showinfo('Agrodem Prepping', 'Browse to the folder that contains required Raster files for temp, prec and radiance')
#file location: r"N:\Agrodem\Irrigation_model\Input_data\Supporting_Layers"
raster_path = filedialog.askdirectory()
raster_files =[]
print ("Reading independent variables...")

for i in os.listdir(raster_path):
    if i.endswith('.tif'):
        raster_files.append(i) 
messagebox.showinfo('Agrodem Prepping','Open the saved shapefile extracted from the input csv file above ')       
shp_filename = filedialog.askopenfilename()


print ("Extracting raster values to points...")

for i in raster_files:
    print("Extracting " + i + " values...")
    src_filename = raster_path + "\\" + i 
    li_values = list()

    src_ds=gdal.Open(src_filename) 
    gt=src_ds.GetGeoTransform()
    rb=src_ds.GetRasterBand(1)
    ds=ogr.Open(shp_filename)
    lyr=ds.GetLayer()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('field_1')
        mx,my=geom.GetX(), geom.GetY()  #coord in map units

        #Convert from map to pixel coordinates.
        #Only works for geotransforms with no rotation.
        px = int((mx - gt[0]) / gt[1]) #x pixel
        py = int((my - gt[3]) / gt[5]) #y pixel

        intval=rb.ReadAsArray(px,py,1,1)
        li_values.append([feat_id, intval[0]])
        
    print ("Writing " + i + " values to csv...")
    
    #input to the output folder for generated csv files
    
    csvoutpath = r"C:\Oluchi\Irrigation model\Maize"    
    
    with open(csvoutpath + "\\" + i.split('.')[0] + i.split('.')[1] + '.csv', 'w') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerows(li_values) 
        

Reading independent variables...
Extracting raster values to points...
Extracting elevation.tif values...
Writing elevation.tif values to csv...
Extracting gyga_af_agg_erzd_tawcpf23mm__m_1km.tif values...
Writing gyga_af_agg_erzd_tawcpf23mm__m_1km.tif values to csv...
Extracting prec.tif values...
Writing prec.tif values to csv...
Extracting srad.tif values...
Writing srad.tif values to csv...
Extracting tavg.tif values...
Writing tavg.tif values to csv...
Extracting wind.tif values...
Writing wind.tif values to csv...
Extracting WTD.tif values...
Writing WTD.tif values to csv...


## Merge csv files with crop

In [13]:
#Import data 

messagebox.showinfo('Agrodem Prepping', 'Open the csv file you in which you exported the geodataframe previously')
file = filedialog.askopenfilename()
agrodem_input = pd.read_csv(file)


In [14]:
csv_files = []

print ("Reading csv files...")

for i in os.listdir(csvoutpath):
    if i.endswith('.csv'):
        csv_files.append(i)  

for i in csv_files:
    print('Reading...'+ i)    
    df_csv = pd.read_csv(csvoutpath + "//" + i, index_col=None, header=None)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].astype(str)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace('[','')
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace(']','')
    columnName = i.split('.')[0]
    
    print("Merging..." + columnName)
    agrodem_input[columnName] = df_csv.iloc[:,1]

Reading csv files...
Reading...BDRICMtif.csv
Merging...BDRICMtif
Reading...BLDtif.csv
Merging...BLDtif
Reading...CLYPPTtif.csv
Merging...CLYPPTtif
Reading...DRAINFAOtif.csv
Merging...DRAINFAOtif
Reading...elevationtif.csv
Merging...elevationtif
Reading...GFSAD-landcovertif.csv
Merging...GFSAD-landcovertif
Reading...gyga_af_agg_erzd_tawcpf23mm__m_1kmtif.csv
Merging...gyga_af_agg_erzd_tawcpf23mm__m_1kmtif
Reading...landcovertif.csv
Merging...landcovertif
Reading...MODIS-Band2tif.csv
Merging...MODIS-Band2tif
Reading...modis-evitif.csv
Merging...modis-evitif
Reading...modis-ndvitif.csv
Merging...modis-ndvitif
Reading...ORCDRCtif.csv
Merging...ORCDRCtif
Reading...PHIHOXtif.csv
Merging...PHIHOXtif
Reading...prectif.csv
Merging...prectif
Reading...sradtif.csv
Merging...sradtif
Reading...SRTM-elevationtif.csv
Merging...SRTM-elevationtif
Reading...tavgtif.csv
Merging...tavgtif
Reading...TEXMHTtif.csv
Merging...TEXMHTtif
Reading...wc20_30s_prec_01.csv
Merging...wc20_30s_prec_01
Reading...wc20_30

In [15]:
# Define output path
# Overwriting the csv file
#path = r"N:\Agrodem\Irrigation_model\Output_data\agrodem_input"
path = r"C:\AgriDem\agrodem_preprocessing\Agrodem_prepping_output"
shpname = "Maize_Benin_1km_trial.csv"

#maize
crop_df.to_csv(os.path.join(path,"{c}".format(c=shpname)))

In [16]:
agrodem_input.columns

Index(['Unnamed: 0', 'Pixel', 'State', 'lon', 'lat', 'Crop', 'Fraction',
       'harv_area', 'sw_depth', 'sw_dist',
       ...
       'wc20_30s_wind_05', 'wc20_30s_wind_06', 'wc20_30s_wind_07',
       'wc20_30s_wind_08', 'wc20_30s_wind_09', 'wc20_30s_wind_10',
       'wc20_30s_wind_11', 'wc20_30s_wind_12', 'windtif', 'WTDtif'],
      dtype='object', length=108)

### Updated names of input files for 30s rasters

In [17]:
# Renaming columns as input file requires
agrodem_input.rename(columns={'wc20_30s_prec_01': 'prec_1',
                              'wc20_30s_prec_02': 'prec_2', 
                              'wc20_30s_prec_03': 'prec_3', 
                              'wc20_30s_prec_04': 'prec_4',
                              'wc20_30s_prec_05': 'prec_5', 
                              'wc20_30s_prec_06': 'prec_6', 
                              'wc20_30s_prec_07': 'prec_7',
                              'wc20_30s_prec_08': 'prec_8', 
                              'wc20_30s_prec_09': 'prec_9', 
                              'wc20_30s_prec_10': 'prec_10',
                              'wc20_30s_prec_11': 'prec_11', 
                              'wc20_30s_prec_12': 'prec_12', 
                              'wc20_30s_srad_01': 'srad_1',
                              'wc20_30s_srad_02': 'srad_2', 
                              'wc20_30s_srad_03': 'srad_3', 
                              'wc20_30s_srad_04': 'srad_4',
                              'wc20_30s_srad_05': 'srad_5', 
                              'wc20_30s_srad_06': 'srad_6', 
                              'wc20_30s_srad_07': 'srad_7',
                              'wc20_30s_srad_08': 'srad_8', 
                              'wc20_30s_srad_09': 'srad_9', 
                              'wc20_30s_srad_10': 'srad_10',
                              'wc20_30s_srad_11': 'srad_11', 
                              'wc20_30s_srad_12': 'srad_12', 
                              'wc20_30s_tavg_01': 'tavg_1',
                              'wc20_30s_tavg_02': 'tavg_2', 
                              'wc20_30s_tavg_03': 'tavg_3', 
                              'wc20_30s_tavg_04': 'tavg_4',
                              'wc20_30s_tavg_05': 'tavg_5', 
                              'wc20_30s_tavg_06': 'tavg_6', 
                              'wc20_30s_tavg_07': 'tavg_7',
                              'wc20_30s_tavg_08': 'tavg_8',
                              'wc20_30s_tavg_09': 'tavg_9', 
                              'wc20_30s_tavg_10': 'tavg_10',
                              'wc20_30s_tavg_11': 'tavg_11', 
                              'wc20_30s_tavg_12': 'tavg_12', 
                              'wc20_30s_tmax_01': 'tmax_1',
                              'wc20_30s_tmax_02': 'tmax_2', 
                              'wc20_30s_tmax_03': 'tmax_3', 
                              'wc20_30s_tmax_04': 'tmax_4',
                              'wc20_30s_tmax_05': 'tmax_5', 
                              'wc20_30s_tmax_06': 'tmax_6', 
                              'wc20_30s_tmax_07': 'tmax_7',
                              'wc20_30s_tmax_08': 'tmax_8', 
                              'wc20_30s_tmax_09': 'tmax_9', 
                              'wc20_30s_tmax_10': 'tmax_10',
                              'wc20_30s_tmax_11': 'tmax_11', 
                              'wc20_30s_tmax_12': 'tmax_12',
                              'wc20_30s_tmin_01': 'tmin_1',
                              'wc20_30s_tmin_02': 'tmin_2', 
                              'wc20_30s_tmin_03': 'tmin_3', 
                              'wc20_30s_tmin_04': 'tmin_4',
                              'wc20_30s_tmin_05': 'tmin_5', 
                              'wc20_30s_tmin_06': 'tmin_6', 
                              'wc20_30s_tmin_07': 'tmin_7',
                              'wc20_30s_tmin_08': 'tmin_8', 
                              'wc20_30s_tmin_09': 'tmin_9', 
                              'wc20_30s_tmin_10': 'tmin_10',
                              'wc20_30s_tmin_11': 'tmin_11', 
                              'wc20_30s_tmin_12': 'tmin_12', 
                              'wc20_30s_wind_01': 'wind_1',
                              'wc20_30s_wind_02': 'wind_2', 
                              'wc20_30s_wind_03': 'wind_3', 
                              'wc20_30s_wind_04': 'wind_4',
                              'wc20_30s_wind_05': 'wind_5', 
                              'wc20_30s_wind_06': 'wind_6', 
                              'wc20_30s_wind_07': 'wind_7',
                              'wc20_30s_wind_08': 'wind_8', 
                              'wc20_30s_wind_09': 'wind_9', 
                              'wc20_30s_wind_10': 'wind_10',
                              'wc20_30s_wind_11': 'wind_11', 
                              'wc20_30s_wind_12': 'wind_12',
                              'gyga_af_agg_erzd_tawcpf23mm__m_1kmtif': 'awsc',
                              'elevationtif': 'elevation',
                              'WTDtif':'gw_depth'}, inplace=True)


In [18]:
agrodem_input.columns

Index(['Unnamed: 0', 'Pixel', 'State', 'lon', 'lat', 'Crop', 'Fraction',
       'harv_area', 'sw_depth', 'sw_dist',
       ...
       'wind_5', 'wind_6', 'wind_7', 'wind_8', 'wind_9', 'wind_10', 'wind_11',
       'wind_12', 'windtif', 'gw_depth'],
      dtype='object', length=108)

In [19]:
agrodem_input.head()

,Unnamed: 0,Pixel,State,lon,lat,Crop,Fraction,harv_area,sw_depth,sw_dist,...,wind_5,wind_6,wind_7,wind_8,wind_9,wind_10,wind_11,wind_12,windtif,gw_depth
0,0,0,Banikoara,2.33763,11.65793,Maize,0.0,0.0,247,0.345832,...,2.6,2.5,2.7,2.1,1.8,1.7,2.1,2.6,0.,3.5370164
1,1,1,Banikoara,2.34662,11.65793,Maize,0.0,0.0,247,0.353748,...,2.7,2.5,2.6,2.1,1.8,1.7,1.9,2.6,0.,0.9739974
2,2,2,Banikoara,2.35560,11.65793,Maize,0.0,0.0,250,1.051088,...,2.6,2.9,2.9,2.1,1.8,1.7,2.,2.6,0.,-0.
3,3,3,Banikoara,2.36458,11.65793,Maize,0.0,0.0,258,1.657820,...,2.6,2.6,2.7,2.1,1.8,1.7,1.9,2.6,0.,0.10699993
4,4,4,Banikoara,2.37357,11.65793,Maize,0.0,0.0,257,1.657937,...,2.7,2.6,2.7,2.1,1.9,1.7,1.8,2.6,0.,2.4400187


In [20]:
# Add geometry and convert to spatial dataframe in source CRS
agrodem_input['geometry'] = list(zip(agrodem_input['lon'], agrodem_input['lat']))
agrodem_input['geometry'] = agrodem_input['geometry'].apply(Point)
agrodem_input = gpd.GeoDataFrame(agrodem_input, geometry='geometry', crs={'init': 'epsg:4326'})

#Reproject data in to Ordnance Survey GB coordinates
agrodem_input = agrodem_input.to_crs({'init': 'epsg:3395'})

In [21]:
agrodem_input.head()

,Unnamed: 0,Pixel,State,lon,lat,Crop,Fraction,harv_area,sw_depth,sw_dist,...,wind_5,wind_6,wind_7,wind_8,wind_9,wind_10,wind_11,wind_12,windtif,gw_depth
0,0,0,Banikoara,2.33763,11.65793,Maize,0.0,0.0,247,0.345832,...,2.6,2.5,2.7,2.1,1.8,1.7,2.1,2.6,0.,3.5370164
1,1,1,Banikoara,2.34662,11.65793,Maize,0.0,0.0,247,0.353748,...,2.7,2.5,2.6,2.1,1.8,1.7,1.9,2.6,0.,0.9739974
2,2,2,Banikoara,2.35560,11.65793,Maize,0.0,0.0,250,1.051088,...,2.6,2.9,2.9,2.1,1.8,1.7,2.,2.6,0.,-0.
3,3,3,Banikoara,2.36458,11.65793,Maize,0.0,0.0,258,1.657820,...,2.6,2.6,2.7,2.1,1.8,1.7,1.9,2.6,0.,0.10699993
4,4,4,Banikoara,2.37357,11.65793,Maize,0.0,0.0,257,1.657937,...,2.7,2.6,2.7,2.1,1.9,1.7,1.8,2.6,0.,2.4400187


In [22]:
droping_cols = ["Unnamed: 0","geometry"]
agrodem_input.drop(droping_cols, axis=1, inplace=True)

## Exporting gdf into csv (or shapefile, gpkg as needed)

In [24]:
#gpkg
#agrodem_input.to_file("Zambezia_1km.gpkg", layer='Maize_Inputfile', driver="GPKG")

#shp
#agrodem_input.to_file("Moz_250m_Maize_190920.shp")

# Define output path
#path = r"C:\Oluchi\Irrigation model\Output_data\agrodem_input\Final_input_files"
path = r"C:\AgriDem\agrodem_preprocessing\Agrodem_prepping_output"
csvname = "agrodem_input_maize_1km_trial.csv"

#maize
agrodem_input.to_csv(os.path.join(path,"{c}".format(c=csvname)), index=False)